# DS24 Deep Learning – Kunskapskontroll 2

Detta är min inlämning för Kunskapskontroll 2 i kursen DS24 Deep Learning.

Notebooken innehåller:
- Del 1: Teoretiska frågor
- Del 2: Enkel RAG-chattbot
- Självutvärdering

## Del 1 – Teoretiska frågor

**1. Hur är AI, Maskininlärning och Deep Learning relaterat?**  
- **AI:** Övergripande område för intelligenta system.  
- **Maskininlärning (ML):** Delmängd av AI, modeller lär sig från data.  
- **Deep Learning (DL):** Delmängd av ML, bygger på djupa neurala nätverk.  

**2. Hur är TensorFlow och Keras relaterat?**  
Keras är ett högnivå-API som använder TensorFlow som backend.  

**3. Vad är en parameter? Vad är en hyperparameter?**  
- Parameter: vikter/bias i modellen som lärs.  
- Hyperparameter: förinställda värden som styr träningen.  

**4. Tränings-, validerings- och testdataset**  
- Träning: uppdatera parametrar.  
- Validering: modellval, undvika överträning.  
- Test: slutlig kontroll.  

**5. Förklarad kod**  
Ett neuralt nät för binär klassificering med ReLU och sigmoid, dropout som regularisering, early stopping för att sluta träna i tid.  

**6. Syftet med regularisering**  
Förhindra överträning, förbättra generalisering.  

**7. Dropout**  
Stänger av slumpmässiga neuroner under träning.  

**8. Early stopping**  
Avbryter träning när valideringsförlusten slutar förbättras.  

**9. Nätverk för bildanalys**  
CNN.  

**10. CNN – översikt**  
Konvolutioner extraherar mönster, pooling minskar dimensioner, djupare lager hittar komplexa drag.  

**11. `model.save` och `load_model`**  
Sparar och laddar modeller.  

**12. CPU och GPU**  
CPU: allmän, sekventiell. GPU: parallell, snabbare för deep learning.  

## Del 2 – Enkel RAG-chattbot

Denna chattbot använder dokument i `docs/`-mappen. Frågor besvaras utifrån innehållet.
En exempelfil (`personalhandbok.txt`) finns redan där.

In [ ]:

# Enkel dokumentladdning och sökning
import os, re, numpy as np
from pathlib import Path

DOCS_DIR = Path("docs")
DOCS_DIR.mkdir(exist_ok=True)

# Ladda dokument
def load_corpus(docs_dir=DOCS_DIR):
    corpus = []
    for p in docs_dir.glob("**/*.txt"):
        text = p.read_text(encoding="utf-8", errors="ignore")
        corpus.append({"path": str(p), "text": text})
    return corpus

corpus = load_corpus()
if not corpus:
    sample = "Exempel: Alla anställda har rätt till 25 semesterdagar per år."
    (DOCS_DIR/"personalhandbok.txt").write_text(sample, encoding="utf-8")
    corpus = load_corpus()

# Skapa enkla representationer (bag-of-words)
texts = [doc["text"].lower() for doc in corpus]
vocab = {}
for t in texts:
    for tok in re.findall(r"\w+", t):
        vocab[tok] = vocab.get(tok, 0) + 1
vocab_list = list(vocab.keys())
idx = {w:i for i,w in enumerate(vocab_list)}
doc_vectors = np.zeros((len(texts), len(vocab_list)), dtype=np.float32)
for i,t in enumerate(texts):
    for tok in re.findall(r"\w+", t):
        j = idx.get(tok)
        if j is not None:
            doc_vectors[i,j] += 1
doc_vectors = doc_vectors / (np.linalg.norm(doc_vectors, axis=1, keepdims=True) + 1e-9)

def embed_query(q):
    v = np.zeros((len(vocab_list),), dtype=np.float32)
    for tok in re.findall(r"\w+", q.lower()):
        j = idx.get(tok)
        if j is not None:
            v[j] += 1
    return v / (np.linalg.norm(v) + 1e-9)

def search(query, k=1):
    qv = embed_query(query)
    sims = doc_vectors @ qv
    idxs = np.argsort(-sims)[:k]
    return [(corpus[i]["path"], corpus[i]["text"], float(sims[i])) for i in idxs]

# Exempel
print(search("Hur många semesterdagar har man rätt till?"))


In [ ]:

# Enkel interaktiv chatt
while True:
    q = input("Fråga (Enter för att avsluta): ").strip()
    if not q:
        print("Avslutar.")
        break
    results = search(q, k=1)
    for path,text,score in results:
        print(f"--- Svar (från {path}) ---\n{text}\n")


## Självutvärdering

- **Roligast:** Att bygga en chattbot som kunde svara på frågor utifrån dokument.  
- **Mest utmanande:** Att förstå hur RAG-flödet fungerar. Jag löste det stegvis.  
- **Betyg:** Jag anser att jag ska ha **Godkänt**, eftersom jag har svarat på alla frågor och gjort en fungerande RAG-chattbot.